In [13]:
import pandas as pd
import random
import os
import numpy as np

from sklearn.preprocessing import LabelEncoder
from sklearn.tree import DecisionTreeRegressor
from xgboost.sklearn import XGBRegressor
from sklearn.model_selection import GridSearchCV

In [14]:
def seed_everything(seed):
    random.seed(seed)
    os.environ['PYTHONHASHSEED'] = str(seed)
    np.random.seed(seed)

seed_everything(42) # Seed 고정

In [15]:
train = pd.read_csv('./data/train.csv')
test = pd.read_csv('./data/test.csv')

In [16]:
train.head()

,ID,추석까지 남은 기간(주),쇼핑몰 구분,가격(원),프로모션 여부,도시 유형,지역 유형,쇼핑몰 유형,선물 유형,수요량
0,TRAIN_0000,1,쇼핑몰 15,212000,0,도시 6,지역 1,쇼핑몰 유형 2,명절혼합과일선물세트,28
1,TRAIN_0001,2,쇼핑몰 72,113000,0,도시 21,지역 1,쇼핑몰 유형 1,발효홍삼선물세트,27
2,TRAIN_0002,0,쇼핑몰 15,67000,0,도시 6,지역 1,쇼핑몰 유형 2,실속스팸선물세트,769
3,TRAIN_0003,1,쇼핑몰 13,206000,0,도시 12,지역 3,쇼핑몰 유형 1,자연산프리미엄버섯선물세트,27
4,TRAIN_0004,1,쇼핑몰 65,140000,0,도시 16,지역 2,쇼핑몰 유형 2,자연산새우선물세트,337


In [17]:
train_x = train.drop(['ID', '수요량'], axis=1)
train_y = train['수요량']

test_x = test.drop(['ID'], axis=1)

In [18]:
ordinal_features = ['쇼핑몰 구분', '도시 유형', '지역 유형', '쇼핑몰 유형', '선물 유형']

for feature in ordinal_features:
    le = LabelEncoder()
    le = le.fit(train_x[feature])
    train_x[feature] = le.transform(train_x[feature])
    
    # train데이터에서 존재하지 않았던 값이 test 데이터에 존재할 수도 있습니다.
    # 따라서 test 데이터를 바로 변형시키지 않고 고유값을 확인후 test 데이터를 변환합니다.
    # Data Leakage를 발생시키지 않기 위함이니, 반드시 주의해주세요.
    
    for label in np.unique(test_x[feature]):
        if label not in le.classes_:
            le.classes_ = np.append(le.classes_, label)
    
    test_x[feature] = le.transform(test_x[feature])

In [19]:
train_x.head()

,추석까지 남은 기간(주),쇼핑몰 구분,가격(원),프로모션 여부,도시 유형,지역 유형,쇼핑몰 유형,선물 유형
0,1,6,212000,0,20,0,1,9
1,2,69,113000,0,13,0,0,15
2,0,6,67000,0,20,0,1,22
3,1,4,206000,0,3,2,0,32
4,1,61,140000,0,7,1,1,30


In [20]:
# Define the parameter grid for Grid Search
param_grid = {
    'n_estimators': [50, 100, 200],
    'max_depth': [3, 5, 7],
    'gamma': [0, 0.1, 0.2],
    'learning_rate': [0.01, 0.05, 0.1],
    'reg_lambda': [0.1, 1, 10]
}

# Initialize XGBoost Regressor
reg = XGBRegressor(random_state=42)

grid_search = GridSearchCV(reg, param_grid, scoring='neg_mean_squared_error', cv=5, verbose=1)

grid_search.fit(train_x, train_y)

best_params = grid_search.best_params_
best_reg = grid_search.best_estimator_

print("Best Parameters:", best_params)

best_reg.fit(train_x, train_y)

Fitting 5 folds for each of 243 candidates, totalling 1215 fits


Best Parameters: {'gamma': 0, 'learning_rate': 0.1, 'max_depth': 5, 'n_estimators': 200, 'reg_lambda': 0.1}


XGBRegressor(base_score=None, booster=None, callbacks=None,
             colsample_bylevel=None, colsample_bynode=None,
             colsample_bytree=None, early_stopping_rounds=None,
             enable_categorical=False, eval_metric=None, feature_types=None,
             gamma=0, gpu_id=None, grow_policy=None, importance_type=None,
             interaction_constraints=None, learning_rate=0.1, max_bin=None,
             max_cat_threshold=None, max_cat_to_onehot=None,
             max_delta_step=None, max_depth=5, max_leaves=None,
             min_child_weight=None, missing=nan, monotone_constraints=None,
             n_estimators=200, n_jobs=None, num_parallel_tree=None,
             predictor=None, random_state=42, ...)

In [23]:
print(best_reg.predict(test_x)[:5])
print()

print('R2 : ', best_reg.score(train_x, train_y))
print()

for i, feature in enumerate(train_x.columns):
    print('{} : {}'.format(feature, best_reg.feature_importances_[i]))

[205.88826   48.663708 303.78076  217.23349  259.2003  ]

R2 :  0.8957685782821228

추석까지 남은 기간(주) : 0.17551857233047485
쇼핑몰 구분 : 0.03488564491271973
가격(원) : 0.09013596177101135
프로모션 여부 : 0.33119282126426697
도시 유형 : 0.052709225565195084
지역 유형 : 0.12610338628292084
쇼핑몰 유형 : 0.08177132904529572
선물 유형 : 0.10768307000398636


In [24]:
submission = pd.read_csv('./data/sample_submission.csv')

In [26]:
preds = best_reg.predict(test_x)
preds

array([ 205.88826 ,   48.663708,  303.78076 , ...,  160.24918 ,
       1205.2538  ,  333.96222 ], dtype=float32)

In [29]:
submission['수요량'] = preds.astype(int)
submission.head()

,ID,수요량
0,TEST_0000,205
1,TEST_0001,48
2,TEST_0002,303
3,TEST_0003,217
4,TEST_0004,259


In [30]:
submission.to_csv('./output/xgboost.csv', index=False)